In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Define the Transformer Autoencoder model
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads):
        super(TransformerAutoencoder, self).__init__()
        # Encoder layers
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        # Decoder layers
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=input_dim, nhead=num_heads, batch_first=True)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.linear = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded, encoded)  # Using encoded output as memory
        return self.linear(decoded)

# Define the Generator model
class Generator(nn.Module):
    def __init__(self, autoencoder_model):
        super(Generator, self).__init__()
        # Use the TransformerAutoencoder model as the generator
        self.autoencoder = autoencoder_model

    def forward(self, x):
        # Generator generates new data instances
        return self.autoencoder(x)

# Define the Discriminator model
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Load data
df = pd.read_csv('creditcard.csv').drop_duplicates().dropna()
data_normalized = (df - df.mean()) / df.std()

# Model parameters
input_dim = 31
hidden_dim = 64
num_layers = 2
num_heads = 1
num_epochs = 5
batch_size = 8
margin = 1.0
p = 0.1

# Initialize models and optimizers
model = TransformerAutoencoder(input_dim, hidden_dim, num_layers, num_heads)
generator = Generator(model)
discriminator = Discriminator(input_dim, hidden_dim)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
ae_optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss functions
criterion_gan = nn.BCELoss()
contrastive_loss = nn.ContrastiveLoss(margin=margin)

# Data preparation
data_normalized_tensor = torch.tensor(data_normalized.values, dtype=torch.float32)
data_loader = DataLoader(data_normalized_tensor, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for data in data_loader:
        batch_size, sequence_length = data.shape
        mask = torch.rand(batch_size, sequence_length) < p
        positive_data = data.clone()
        positive_data[mask] = 0
        negative_data = torch.roll(data, shifts=1, dims=0)
        
        # Train Discriminator
        real_output = discriminator(data)
        real_loss = criterion_gan(real_output, torch.ones_like(real_output))
        fake_data = generator(data)
        fake_output = discriminator(fake_data.detach())
        fake_loss = criterion_gan(fake_output, torch.zeros_like(fake_output))
        d_loss = real_loss + fake_loss
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        trick_output = discriminator(fake_data)
        g_loss = criterion_gan(trick_output, torch.ones_like(trick_output))
        g_loss.backward()
        g_optimizer.step()

        # Train Autoencoder with Contrastive Learning
        ae_optimizer.zero_grad()
        original_recon = model(data)
        positive_recon = model(positive_data)
        negative_recon = model(negative_data)
        pos_loss = contrastive_loss(original_recon, positive_recon, torch.ones(data.size(0)))
        neg_loss = contrastive_loss(original_recon, negative_recon, torch.zeros(data.size(0)))
        ae_loss = pos_loss + neg_loss
        ae_loss.backward()
        ae_optimizer.step()

        print(f'Epoch {epoch+1}, D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}, AE Loss: {ae_loss.item():.4f}')

# Metric calculation function
def calculate_metrics(predicted, actual, threshold=0.1):
    mse = torch.mean((predicted - actual) ** 2).item()
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(predicted - actual)).item()
    correct_predictions = torch.abs(predicted - actual) <= threshold
    accuracy = torch.mean(correct_predictions.float()).item()
    return {"MSE": mse, "RMSE": rmse.item(), "MAE": mae, "Accuracy": accuracy}

# Evaluate metrics
for epoch in range(num_epochs):
    predicted_all = []
    actual_all = []
    for data in data_loader:
        with torch.no_grad():
            predicted = model(data)
            actual = data
            predicted_all.append(predicted)
            actual_all.append(actual)

    predicted_tensor = torch.cat(predicted_all, 0)
    actual_tensor = torch.cat(actual_all, 0)

    metrics = calculate_metrics(predicted_tensor, actual_tensor)
    print(f"Epoch {epoch+1} Metrics:", metrics)

# Anomaly detection function
def detect_anomalies(data_loader, model, threshold):
    anomalies = []
    for data in data_loader:
        with torch.no_grad():
            reconstructed = model(data)
            errors = torch.mean((data - reconstructed) ** 2, dim=1)
            anomalies.extend([(i, error.item()) for i, error in enumerate(errors) if error.item() > threshold])
    return anomalies

# Detect anomalies
anomaly_threshold = 0.1
anomalies = detect_anomalies(data_loader, model, anomaly_threshold)

# Print anomalies
print("Anomalies:")
for index, error in anomalies:
    print(f"Index: {index}, Reconstruction Error: {error}")


AttributeError: module 'torch.nn' has no attribute 'ContrastiveLoss'